In [1]:
# !apt-get install -y xvfb python-opengl > /dev/null 2>&1
# !pip install gym pyvirtualdisplay > /dev/null 2>&1

IndentationError: unexpected indent (<ipython-input-1-7401f2c8b590>, line 2)

In [6]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay

In [7]:
from pyvirtualdisplay import Display
# display = Display(visible=0, size=(400, 300))
# display.start()

EasyProcessError: start error <EasyProcess cmd_param=['Xvfb', '-help'] cmd=['Xvfb', '-help'] oserror=[WinError 2] The system cannot find the file specified return_code=None stdout="None" stderr="None" timeout_happened=False>

# **Exercise 1:**

Can you design a dynamic programming based policy for the agent as in assignment 1? If so, design it and demonstrate that it solves the cart pole problem.

**Answer**: No we can't use dynamic programming based policy for the agent since we do not have complete knowledge of environment and thus we can not initialize all the states.

# **Exercise 2:**

Can you design a Monte Carlo based policy for the agent? What ingredients do you require? Explain the design flow, and execute it. Show that it works, or indicate why you can't proceed.

In [8]:
env = gym.make("CartPole-v0")
env.reset()
s, r, done, info = env.step(0)

In [9]:
import random
# Q is a dictionary of dictionary
nepisodes = 200
Q = {}
returns = {}

def pi(s, epsilon = 0.1):
  if (random.random() < epsilon) or (s not in Q):
    return env.action_space.sample()
  else:
    return max(Q[s], key=Q[s].get)

def state_to_string(s):
  result = ''
  for i, item in enumerate(s):
    if i != len(s)  - 1:
      result += str(round(item, 4)) + '_'
    else:
      result += str(round(item, 4))
  return result

def generate_episode():
  s = env.reset()
  s = state_to_string(s)
  E = []
  for i in range(nepisodes):
    action = pi(s)
    next_s, next_r, done, info = env.step(action)
    E.append({"state": s, "action": action, "reward": next_r})
    screen = env.render(mode='rgb_array')
    if done:
      break
    
    s = state_to_string(next_s)
  return E

def update_returns(s, a, G):
  if s in returns and a in returns[s]:
    count, value = returns[s][a]['count'], returns[s][a]['value']
    returns[s][a]['value'] = (count * value + G)/(count + 1)
    returns[s][a]['count'] += 1
  elif s in returns:
    returns[s][a] = {'value': G, 'count': 1}
  else:
    returns[s] = {a: {'value': G, 'count': 1}}

def update_q():
  for s in returns.keys():
    Q[s] = {a: returns[s][a]['value'] for a in returns[s].keys()}



decay = 0.8
k = 0
while True:
  k += 1
  print(f"Iteration: {k}")
  E = generate_episode()
  if len(E) == nepisodes:
    break
  G = 0
  for i in range(len(E) - 1, 0, -1):
    s, a, r = E[i]['state'], E[i]['action'], E[i]['reward']
    G = decay * G + r
    update_returns(s, a, G)
    update_q()
print(f"Done training after iteration {k}")
# Test
env = gym.make("CartPole-v0")
s = env.reset()
s = state_to_string(s)
prev_screen = env.render(mode='rgb_array')
plt.imshow(prev_screen)

for i in range(50000):
  print(f"Iteration: {i}")
  action = pi(s)
  print("step i",i,"action=",action)
  obs, reward, done, info = env.step(action)
  s = state_to_string(obs)
  print("obs=",obs,"reward=",reward,"done=",done,"info=",info)
  screen = env.render(mode='rgb_array')
  
  plt.imshow(screen)
  ipythondisplay.clear_output(wait=True)
  ipythondisplay.display(plt.gcf())

  if done:
    break
    
# ipythondisplay.clear_output(wait=True)
env.close()
print("Iterations that were run:",i)
end = time.time()
print(end - start)

KeyboardInterrupt: 